Question 1 (5 Marks)



In [1]:
!pip list

Package           Version
----------------- -----------
asttokens         3.0.0
comm              0.2.2
debugpy           1.8.14
decorator         5.2.1
exceptiongroup    1.2.2
executing         2.2.0
ipykernel         6.29.5
ipython           8.35.0
jedi              0.19.2
jupyter_client    8.6.3
jupyter_core      5.7.2
matplotlib-inline 0.1.7
nest-asyncio      1.6.0
packaging         24.2
parso             0.8.4
pexpect           4.9.0
pip               23.0.1
platformdirs      4.3.7
prompt_toolkit    3.0.50
psutil            7.0.0
ptyprocess        0.7.0
pure_eval         0.2.3
Pygments          2.19.1
python-dateutil   2.9.0.post0
pyzmq             26.4.0
setuptools        65.5.0
six               1.17.0
stack-data        0.6.3
tornado           6.4.2
traitlets         5.14.3
typing_extensions 4.13.2
wcwidth           0.2.13

[notice] A new release of pip is available: 23.0.1 -> 25.0.1
[notice] To update, run: pip install --upgrade pip


In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class CustomCNN(nn.Module):
    def __init__(self, 
                 channels=3,
                 num_filters=64,
                 kernel_size=3,
                 activation_fn=nn.ReLU,
                 dense_neurons=256,
                 num_classes=10):
        super(CustomCNN, self).__init__()

        self.activation_fn = activation_fn

        layers = []
        current_channels = channels
        for _ in range(5):
            layers.append(nn.Conv2d(current_channels, num_filters, kernel_size=kernel_size, padding=kernel_size // 2))
            layers.append(activation_fn())
            layers.append(nn.MaxPool2d(kernel_size=2))
            current_channels = num_filters

        self.conv_layers = nn.Sequential(*layers)

        # ✅ Corrected this line
        self.flattened_size = self._get_flattened_size(channels)

        self.fc1 = nn.Linear(self.flattened_size, dense_neurons)
        self.fc2 = nn.Linear(dense_neurons, num_classes)

    def _get_flattened_size(self, in_channels):
        dummy_input = torch.zeros(1, in_channels, 224, 224)
        with torch.no_grad():
            output = self.conv_layers(dummy_input)
        return output.view(1, -1).shape[1]

    def forward(self, x):
        x = self.conv_layers(x)
        x = x.view(x.size(0), -1)
        x = self.activation_fn()(self.fc1(x))
        x = self.fc2(x)
        return x


# Dummy input (simulating a 224x224 RGB image)
dummy_image = torch.randn(1, 3, 224, 224)
print(dummy_image)

# ✅ Instantiate the model
model = CustomCNN(
    channels=3,
    num_filters=64,
    kernel_size=3,
    activation_fn=nn.ReLU,
    dense_neurons=256,
    num_classes=10
)

# ✅ Forward pass
output = model(dummy_image)
print("Output shape:", output.shape)  # Output shape: torch.Size([1, 10])


/mnt/e_disk/ch24s016/da6401_assignment2/venv/lib/python3.10/site-packages/torch/_subclasses/functional_tensor.py:275: UserWarning: Failed to initialize NumPy: No module named 'numpy' (Triggered internally at /pytorch/torch/csrc/utils/tensor_numpy.cpp:81.)
  cpu = _conversion_method_template(device=torch.device("cpu"))


tensor([[[[-0.4345, -0.3812, -0.8483,  ..., -1.1933,  1.0156, -0.9892],
          [-1.0213,  0.3691, -0.5498,  ..., -0.7659,  0.5248,  0.8751],
          [-1.3958, -0.8850,  0.7639,  ..., -0.0196,  0.6186, -1.0443],
          ...,
          [ 0.3500, -1.0127,  1.3175,  ..., -0.1168, -1.1132, -2.1547],
          [ 1.8374,  1.7425,  0.9138,  ...,  0.3981, -1.4344, -0.0211],
          [ 2.8739, -0.0033,  0.9340,  ...,  1.2583,  0.7334,  1.1437]],

         [[-0.0101, -0.8702,  0.8240,  ...,  0.5718,  1.4387, -0.5587],
          [ 0.6007,  0.4935, -0.0058,  ..., -1.4167, -0.3241,  0.5273],
          [ 0.1731, -0.0257,  1.3791,  ..., -1.1895,  0.2785,  0.8173],
          ...,
          [-0.4200,  1.3456,  0.5558,  ...,  0.3346, -0.4340,  0.4964],
          [ 1.2490,  1.5329, -1.4666,  ..., -0.0402,  1.1058,  0.2795],
          [ 0.1511,  0.4524, -1.7334,  ...,  0.8952,  0.5167, -0.3186]],

         [[-0.3119,  0.0283, -0.2561,  ...,  0.9748, -0.0306, -0.7446],
          [-0.1050,  0.5038,  

In [ ]:
import os
import torch
import torchvision.transforms as transforms
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader, Subset
import matplotlib.pyplot as plt

# ✅ Transform (match your model input size)
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor()
])

# ✅ Load dataset (folder structure must be class-wise)
dataset_path = 'naturalist_dataset'  # <- your dataset folder
dataset = ImageFolder(root=dataset_path, transform=transform)
class_names = dataset.classes

# ✅ Take only a few samples per class
samples_per_class = 2
indices = []
class_counter = {cls_idx: 0 for cls_idx in range(len(class_names))}

for idx, (_, label) in enumerate(dataset):
    if class_counter[label] < samples_per_class:
        indices.append(idx)
        class_counter[label] += 1
    if all(v >= samples_per_class for v in class_counter.values()):
        break

# ✅ Wrap in DataLoader
subset = Subset(dataset, indices)
dataloader = DataLoader(subset, batch_size=8, shuffle=False)

# ✅ Load your model
model = model.to('cuda')
images = images.to('cuda')
model = CustomCNN(num_classes=len(class_names))
model.load_state_dict(torch.load('your_model_weights.pth', map_location='cpu'))
model.eval()

ModuleNotFoundError: No module named 'torchvision'

In [ ]:
def show_predictions(model, dataloader, class_names):
    with torch.no_grad():
        for images, labels in dataloader:
            outputs = model(images)
            _, preds = torch.max(outputs, 1)

            for i in range(images.size(0)):
                img = images[i].permute(1, 2, 0).numpy()
                true_label = class_names[labels[i]]
                predicted_label = class_names[preds[i]]

                plt.imshow(img)
                plt.title(f"Predicted: {predicted_label}\nActual: {true_label}")
                plt.axis('off')
                plt.show()

# ✅ Show results
show_predictions(model, dataloader, class_names)